<a href="https://colab.research.google.com/github/Amir-D-Shadow/Google-Colab/blob/main/preprocess_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random
from numba import jit
import statistics

In [2]:
a = [(6,989),(546,2),(456,4)]
b = random.choice(a)
print(type(b))

<class 'tuple'>


In [3]:
a = np.random.randn(6,9,8)
b = a[:,:,np.newaxis,:]
b.shape

(6, 9, 1, 8)

In [4]:
a = [1,6,9,8,33]
b = statistics.mean(a)
print(type(b))

<class 'float'>


In [5]:
#@jit(nopython=True)
def best_anchor(box,anchors):
  """
  box -- (h,w)

  anchor -- List [(h1,w1),(h2,w2),.....,(hk,wk)]

  return class
  """
  max_iou = 0
  max_index = 0
  index = 0

  for anchor_hw in anchors:

    min_h = min(box[0],anchor_hw[0])
    min_w = min(box[1],anchor_hw[1])

    intersection_area = min_h * min_w

    union_area = box[0] * box[1] + anchor_hw[0] * anchor_hw[1] - intersection_area

    cur_iou = intersection_area / (union_area + 0.000000001)

    if cur_iou > max_iou:

      max_iou = cur_iou

      max_index = index

    index = index + 1
    
  return max_index

In [6]:
def update_anchor_x(class_list,anchors_box):

  """
  class_list -- list[(h1,w1),(h2,w2),....]
  """

  sum_w = 0
  sum_h = 0

  for sample in class_list:

    sum_h = sum_h + sample[0]
    sum_w = sum_w + sample[1]

  m = len(class_list)

  if m == 0:

    return anchors_box[0],anchors_box[1]

  mean_h = sum_h/m
  mean_w = sum_w/m

  return mean_h,mean_w


In [7]:
def K_mean(inputs,K=6,threshold=0.2,max_iterations=10000):

  """
  inputs -- list[(h1,w1),(h2,w2),....]
  """
  #Initialize center
  anchors = []

  for i in range(K):

    anchors.append(random.choice(inputs))

  iterations_i = 0

  #Calculate K-means
  while iterations_i < max_iterations:

    class_collections = [[] for i in range(K)]

    for box in inputs:

      class_id = best_anchor(box,anchors)

      class_collections[class_id].append(box)

    #used for checking threshold
    sum_diff = 0

    #update anchors
    for class_idx,sample_x in enumerate(class_collections):

        new_h,new_w = update_anchor_x(sample_x,anchors[class_idx])

        #sum the changes for w and h of new anchor box
        sum_diff = sum_diff + ((new_h-anchors[class_idx][0])**2 + (new_w - anchors[class_idx][1])**2)**(0.5)

        #update anchor box
        anchors[class_idx] = (new_h,new_w)

    if sum_diff <= threshold :

      return anchors

  return anchors

In [8]:
a = [(i,j) for i,j in enumerate(range(100))]
K_mean(a)

[(33.05555555555556, 33.05555555555556),
 (21.0, 21.0),
 (3.5, 3.5),
 (56.0, 56.0),
 (84.0, 84.0),
 (11.0, 11.0)]

In [9]:
from sklearn.cluster import KMeans

res = KMeans(n_clusters=6)
res.fit(a)

label = res.predict(a)
print(label)

[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
